In [1]:
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
# %jaxtyping.typechecker beartype.beartype

env: JAX_PLATFORM_NAME=cpu


In [2]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from tqdm.notebook import trange

from jaxtyping import Array, Bool, Float

from chaogatenn.chaogate import ChaoGate
from chaogatenn.maps import (
    LogisticMap,
    HenonMap,
    DuffingMap,
    LorenzMap,
    RosslerMap,
    ChenMap,
    RosslerHyperchaosMap,
)
from chaogatenn.utils import grad_norm

In [3]:
# Training data for the AND gate
X = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=bool)  # Input combinations
AND_Y = jnp.array([0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y = jnp.array([0, 1, 1, 1], dtype=bool)  # OR gate output
XOR_Y = jnp.array([0, 1, 1, 0], dtype=bool)  # XOR
NAND_Y = jnp.array([1, 1, 1, 0], dtype=bool)  # NAND
NOR_Y = jnp.array([1, 0, 0, 0], dtype=bool)  # NOR
XNOR_Y = jnp.array([1, 0, 0, 1], dtype=bool)  # XNOR
Y = NOR_Y

In [4]:
# Map = LogisticMap(a=4.0)
Map = HenonMap(a=1.4, b=0.3)
# Map = LorenzMap()
# Map = DuffingMap(
#     alpha=1.0, beta=1.0, delta=0.02, gamma=8.0, omega=0.5, dt=0.01, steps=1000
# )
# Map = ChenMap(dt=1e-5, steps=100)
# Map = RosslerHyperchaosMap()

In [5]:
Map(2)  # type: ignore

-8.0

In [6]:
DELTA, X0, X_THRESHOLD = jax.random.normal(jax.random.PRNGKey(0), (3,))
chao_gate = ChaoGate(DELTA=DELTA, X0=X0, X_THRESHOLD=X_THRESHOLD, Map=Map)

In [7]:
[chao_gate(x) for x in X]

[Array(4.0121733e-05, dtype=float32),
 Array(0.0005, dtype=float32),
 Array(0.0005, dtype=float32),
 Array(0.00469383, dtype=float32)]

In [8]:
@eqx.filter_value_and_grad()
def compute_loss(
    chao_gate: ChaoGate, x: Bool[Array, "batch 2"], y: Bool[Array, "batch"]
) -> Float[Array, ""]:  # noqa: F821
    pred = jax.vmap(chao_gate)(x)
    # binary cross entropy + weak constraints
    return (
        50
        * (-jnp.mean(y * jnp.log(pred + 1e-15) + (1 - y) * jnp.log(1 - pred + 1e-15)))
        + 10 * chao_gate.DELTA**2
        + 3 * jnp.exp(-chao_gate.X0)
        + 3 * jnp.exp(-chao_gate.X_THRESHOLD)
        + 10 * jnp.exp(-chao_gate.DELTA)
    )

In [9]:
@eqx.filter_jit
def make_step(
    model: ChaoGate,
    x: Bool[Array, "dim 2"],
    y: Bool[Array, "dim"],  # noqa: F821
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> (Float[Array, "dim"], ChaoGate, optax.OptState):  # type: ignore  # noqa: F821
    loss, grads = compute_loss(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    # jax.debug.print(f"{grads, updates}")
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state

In [10]:
optim = optax.adabelief(3e-4)
opt_state = optim.init(eqx.filter(chao_gate, eqx.is_inexact_array))

In [11]:
epochs = 10_000

for epoch in trange(epochs):
    loss, chao_gate, opt_state = make_step(chao_gate, X, Y, optim, opt_state)  # type: ignore
    _, grads = compute_loss(chao_gate, X, Y)
    grad_norm_value = grad_norm(grads)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}, Grad Norm: {grad_norm_value}")

    # if loss < 1e-3:
    #     break

  0%|          | 0/10000 [00:00<?, ?it/s]

Epoch 0, Loss: 156.1767578125, Grad Norm: 188.01612854003906
Epoch 10, Loss: 155.47047424316406, Grad Norm: 187.6153564453125
Epoch 20, Loss: 154.4741973876953, Grad Norm: 187.04917907714844
Epoch 30, Loss: 153.223876953125, Grad Norm: 186.32644653320312
Epoch 40, Loss: 151.7606201171875, Grad Norm: 185.4607391357422
Epoch 50, Loss: 150.1172332763672, Grad Norm: 184.46676635742188
Epoch 60, Loss: 148.318603515625, Grad Norm: 183.35877990722656
Epoch 70, Loss: 146.38455200195312, Grad Norm: 182.149658203125
Epoch 80, Loss: 144.33145141601562, Grad Norm: 180.85072326660156
Epoch 90, Loss: 142.17333984375, Grad Norm: 179.47195434570312
Epoch 100, Loss: 139.9225311279297, Grad Norm: 178.02142333984375
Epoch 110, Loss: 137.5898895263672, Grad Norm: 176.50546264648438
Epoch 120, Loss: 135.18539428710938, Grad Norm: 174.92886352539062
Epoch 130, Loss: 132.718017578125, Grad Norm: 173.29507446289062
Epoch 140, Loss: 130.19613647460938, Grad Norm: 171.60716247558594
Epoch 150, Loss: 127.6275482

In [12]:
print("\nTrained ChaoGate Parameters:")
print(
    f"DELTA: {chao_gate.DELTA}, X0: {chao_gate.X0}, X_THRESHOLD: {chao_gate.X_THRESHOLD}"
)


Trained ChaoGate Parameters:
DELTA: -0.5018762350082397, X0: 0.1302088052034378, X_THRESHOLD: 0.12557074427604675


In [13]:
[
    (
        bool(x1.item()),
        bool(x2.item()),
        (
            chao_gate.Map(chao_gate.X0 + x1 * chao_gate.DELTA + x2 * chao_gate.DELTA)
            > chao_gate.X_THRESHOLD
        ).item(),
    )
    for x1, x2 in X
]

[(False, False, True),
 (False, True, False),
 (True, False, False),
 (True, True, False)]

In [14]:
pred_ys = jax.vmap(chao_gate)(X)
num_correct = jnp.sum((pred_ys > 0.5) == Y)
final_accuracy = (num_correct / len(X)).item()
print(f"final_accuracy={final_accuracy}")

final_accuracy=1.0
